## Install Semantic Link Labs Package

In [ ]:
%pip install semantic-link-labs

## Import Packages

In [ ]:
import sempy_labs.report as rep
import base64
import json
import pandas as pd
import hashlib

## Set Report and Workspace ids

In [ ]:
report = 'f173e7cf-8868-4909-b271-5d4730570c04' 
workspace = 'b1c1beef-4463-41b6-a24b-764acf8972f2' 

In [ ]:
files = rep.get_report_definition(report, workspace)
row = files[files["path"] == "report.json"].iloc[0]
decoded = base64.b64decode(row["payload"]).decode("utf-8")
json_obj = json.loads(decoded)

# Function to safely try to parse JSON strings
def try_json_loads(s):
    """Safely attempts to parse a string as JSON, returning the string itself on failure."""
    try:
        # Check if input is a valid string before decoding
        if isinstance(s, str):
            return json.loads(s)
        return s
    except (TypeError, json.JSONDecodeError):
        return s

# Initialize list to hold records
records = []

for section in json_obj.get("sections", []):
    page_name = section.get("displayName", "Unnamed Page")
    visuals = section.get("visualContainers", [])

    for z_order, visual in enumerate(visuals):
        
        # 1. Load config and extract metadata first
        visual_conf = try_json_loads(visual.get("config", "{}"))
        visual_name_user = visual_conf.get("name", "Unnamed Visual")
        single_visual = visual_conf.get("singleVisual", {})
        visual_type = single_visual.get("visualType")
        
        # 2. 🔑 ROBUST UNIQUE ID SEARCH 🔑
        # a. Try 'name' on the visual container (the ideal GUID/ID)
        visual_container_id = visual.get("name")
        
        # b. Fallback: Check if the 'config' object itself has a machine-readable 'name' or 'id'
        if not visual_container_id or visual_container_id == "GUID_Missing_Error":
             visual_container_id = visual_conf.get("name") # Often contains the user-defined name
        
        # c. If still no unique machine ID, use the coordinate/z_order combination
        if not visual_container_id or visual_container_id == visual_name_user:
            # Use a descriptive fallback based on position and type
            # We use z_order as the most reliable unique component if the GUID is missing.
            visual_container_id = f"{visual_type}_Z{z_order}"


        # 3. Extract Coordinates and Layering
        x_pos = visual.get("x", "x_MISSING")
        y_pos = visual.get("y", "y_MISSING")
        
        # 4. Generate the Highly Unique Key
        # Format: PAGE::ID::L_ZORDER_(X,Y)
        visual_id_unique = f"{page_name}::{visual_container_id}::L{z_order}_({x_pos},{y_pos})"
        
        if not visual_type:
            continue
            
        proto_query = single_visual.get("prototypeQuery", {})
        column_props = single_visual.get("columnProperties", {})

        # Build alias → table map
        alias_map = {entry["Name"]: entry["Entity"] for entry in proto_query.get("From", []) if "Name" in entry and "Entity" in entry}

        # Map queryRef → Select entry for resolving expressions
        select_map = {sel.get("Name"): sel for sel in proto_query.get("Select", [])}

        # Iterate over projections (visual roles)
        projections = single_visual.get("projections", {})
        for proj_key, proj_list in projections.items():
            for proj in proj_list:
                query_ref = proj.get("queryRef")
                if not query_ref:
                    continue

                sel = select_map.get(query_ref)
                visual_display_name = column_props.get(query_ref, {}).get("displayName", query_ref)
                
                # Initialize variables
                table = None
                original_model_field = visual_display_name
                type_ = "unknown"
                
                # --- Model Field Logic (Omitted for brevity, but remains unchanged) ---
                if sel:
                    if "Column" in sel:
                        col_info = sel["Column"]
                        alias = col_info.get("Expression", {}).get("SourceRef", {}).get("Source") or \
                                col_info.get("Expression", {}).get("SourceRef", {}).get("Entity")
                        table = alias_map.get(alias, alias)
                        original_model_field = f"'{table}'[{col_info.get('Property')}]" if table and col_info.get('Property') else col_info.get('Property')
                        type_ = "column"
                        
                    elif "Measure" in sel or "Aggregation" in sel:
                        if "Measure" in sel:
                            table_alias = sel["Measure"].get("Expression", {}).get("SourceRef", {}).get("Source")
                            table = alias_map.get(table_alias, table_alias)
                            measure_property = sel["Measure"].get("Property")
                            original_model_field = f"'{table}'[{measure_property}]" if table and measure_property else measure_property
                            type_ = "measure"

                        elif "Aggregation" in sel:
                            agg_column = sel["Aggregation"].get("Expression", {}).get("Column", {})
                            table_alias = agg_column.get("Expression", {}).get("SourceRef", {}).get("Source")
                            table = alias_map.get(table_alias, table_alias)
                            agg_func = sel["Aggregation"].get("Function")
                            column_name = agg_column.get("Property")
                            original_model_field = f"{agg_func}('{table}'[{column_name}])" if table and column_name else f"{agg_func}({column_name})"
                            type_ = "implicit_measure"
                
                if original_model_field == visual_display_name:
                    original_model_field = query_ref

                records.append({
                    "page_name": page_name,
                    "visual_id_unique": visual_id_unique,
                    "visual_name_user": visual_name_user,
                    "visual_type": visual_type,
                    "visual_role": proj_key, 
                    "table": table,
                    "model_field_qualified": original_model_field,
                    "visual_display_name": visual_display_name,
                    "type": type_,
                    "query_ref": query_ref,
                    "x_pos": x_pos,
                    "y_pos": y_pos,
                    "z_order": z_order,
                    "width": visual.get("width"),
                    "height": visual.get("height")
                })

# Create and display the DataFrame
df = pd.DataFrame(records)
# Assuming 'display' function exists in your notebook environment:
display(df)